imports

In [1]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import vectorbtpro as vbt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400


data

In [2]:
sol_data = pd.read_csv('2ySOLdata1h.csv')
sol_data['timestamp'] = pd.to_datetime(sol_data['timestamp'], unit='s')
sol_data.set_index('timestamp', inplace=True)
pd.set_option('future.no_silent_downcasting', True)

In [3]:
data = sol_data.iloc[:, 0:5].copy()
data


,Open,High,Low,Close,Volume
timestamp,,,,,
2022-01-01 04:00:00,172.790,173.130,172.480,172.930,49675
2022-01-01 05:00:00,172.940,173.110,171.470,171.470,68973
2022-01-01 06:00:00,171.490,174.310,171.340,173.220,95595
2022-01-01 07:00:00,173.220,173.740,172.260,172.470,95494
2022-01-01 08:00:00,172.450,173.440,172.140,173.160,57926
...,...,...,...,...,...
2023-12-31 02:00:00,102.718,103.063,101.208,101.411,731765
2023-12-31 03:00:00,101.413,101.850,100.044,100.738,970135
2023-12-31 04:00:00,100.734,100.939,99.635,100.743,858035


In [4]:

data_trimmed = data.copy()
pd.set_option('future.no_silent_downcasting', True)
data_trimmed.loc[:, 'signal'] = 'SignalNone'


# Define window size
window_size = 10

rolling_max = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).max()
rolling_min = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).min()

is_peak = (data_trimmed.loc[:, 'Close'] == rolling_max)

is_low = (data_trimmed.loc[:, 'Close'] == rolling_min) 

# Update signal columns where conditions are met
data_trimmed.loc[is_peak, 'signal'] = 'SignalShort'  # Mark peaks with SignalShort
data_trimmed.loc[is_low, 'signal'] = 'SignalLong'   # Mark lows with SignalLong
df = data_trimmed.copy()
df_filtered = df[df['signal'] != 'SignalNone'].copy()

# Iterate through the DataFrame and adjust the signals
for i in range(1, len(df_filtered)):
    current_signal = df_filtered.iloc[i]['signal']
    previous_signal = df_filtered.iloc[i - 1]['signal']
    current_close = df_filtered.iloc[i]['Close']
    previous_close = df_filtered.iloc[i - 1]['Close']
    
    if current_signal == previous_signal:
        if current_signal == 'SignalLong' and previous_close > current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        elif current_signal != 'SignalLong' and previous_close < current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        else:
            df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'


df.update(df_filtered)

df.loc[:,'signal'] = df.loc[:,'signal'].replace({'SignalLong': 2, 'SignalShort': 0, 'SignalNone': 1})
df = df.ffill()

df['signal'] = df['signal'].astype(float)
long_signals = df['signal'] == 2
short_signals = df['signal'] == 0

scaler = MinMaxScaler(feature_range=(0, 2))
for idx in df.index[:-1]:
    if short_signals.loc[idx]:
        short_index = idx
        next_long_idx = df.loc[idx:].index[long_signals[idx:]].min()
        bear_slice = df.loc[short_index : next_long_idx].copy()
        bear_slice['signal'] = bear_slice['Close']
        signal_values = bear_slice['signal'].values.reshape(-1, 1)
        scaled_signal_values = scaler.fit_transform(signal_values)
        scaled_signal_values_transformed = 2 - (scaled_signal_values)
        bear_slice['signal'] = scaled_signal_values_transformed.flatten()
        df.update(bear_slice)
    elif long_signals.loc[idx]:
        long_index = idx
        next_short_idx = df.loc[idx:].index[short_signals[idx:]].min()
        bull_slice = df.loc[long_index : next_short_idx].copy()
        bull_slice['signal'] = bull_slice['Close']
        signal_values = bull_slice['signal'].values.reshape(-1, 1)
        scaled_signal_values = scaler.fit_transform(signal_values)
        scaled_signal_values_transformed = 2 - (scaled_signal_values)
        bull_slice['signal'] = scaled_signal_values_transformed.flatten()
        df.update(bull_slice)


In [5]:
df_sol=df.copy()

In [6]:
df_sol = vbt.Data.from_data(df)

features = df_sol.run("talib", mavp=vbt.run_arg_dict(periods=14))

df_sol.data['symbol'] = pd.concat([df_sol.data['symbol'], features], axis=1)
# This will drop columns from the DataFrame where all values are NaN
df_sol.data['symbol'] = df_sol.data['symbol'].dropna(axis=1, how='all')


df_sol.data['symbol'] = df_sol.data['symbol'].dropna()
predictor_list = df_sol.data['symbol'].drop('signal', axis=1).columns.tolist()
X = df_sol.data['symbol'][predictor_list]

y = df_sol.data['symbol']['signal']

X.columns = X.columns.astype(str)


train test split

In [7]:
# Split the data into a training set and a test set
# Assuming X is a DataFrame or a NumPy array
from sklearn.model_selection import train_test_split

indices = np.arange(X.shape[0])

# First, split your data into a training+validation set and a separate test set
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size=0.3, shuffle=False)


scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


sequence

In [8]:
timestep = 40

def create_sequences(input_data, timestep):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - timestep):
        seq = input_data[i:(i + timestep)]
        sequences.append(seq)
    return np.array(sequences)

X_train_list = create_sequences(X_train_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
y_train_seq_ar = y_train[timestep:]
y_test_seq_ar = y_test[timestep:]

# Convert to numpy arrays
x_train_ar = np.array(X_train_list)
x_test_ar = np.array(X_test_list)  
y_train_seq = np.array(y_train_seq_ar).reshape(-1, 1)
y_test_seq = np.array(y_test_seq_ar).reshape(-1, 1)


# make training and test sets in torch
x_train_tensor = torch.from_numpy(x_train_ar).type(torch.Tensor)
x_test_tensor = torch.from_numpy(x_test_ar).type(torch.Tensor)
y_train_tensor = torch.from_numpy(y_train_seq).type(torch.Tensor)
y_test_tensor = torch.from_numpy(y_test_seq).type(torch.Tensor)
y_train_tensor.size(),x_train_tensor.size()

(torch.Size([12147, 1]), torch.Size([12147, 40, 178]))

model

In [9]:
# Build model
#####################
input_dim = x_train_tensor.shape[2]
hidden_dim = 32
num_layers = 2 
output_dim = 1


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out
    
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
loss_fn = torch.nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=20, gamma=0.8)


In [10]:
# Train model
#####################
num_epochs = 1000
hist = np.zeros(num_epochs)  

for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    # model.hidden = model.init_hidden()
    
    # Forward pass
    y_train_pred = model(x_train_tensor)


    loss = loss_fn(y_train_pred, y_train_tensor)
    if t % 10 == 0:
        print("Epoch", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()
    scheduler.step()

Epoch 0 MSE:  1.732309103012085
Epoch 10 MSE:  0.3574609160423279
Epoch 20 MSE:  0.33088088035583496
Epoch 30 MSE:  0.3234833776950836
Epoch 40 MSE:  0.32078805565834045
Epoch 50 MSE:  0.32011109590530396
Epoch 60 MSE:  0.31976500153541565
Epoch 70 MSE:  0.3195667266845703
Epoch 80 MSE:  0.31924381852149963
Epoch 90 MSE:  0.3187253773212433
Epoch 100 MSE:  0.31785592436790466
Epoch 110 MSE:  0.3164445459842682
Epoch 120 MSE:  0.3140912652015686
Epoch 130 MSE:  0.3114085793495178
Epoch 140 MSE:  0.30739372968673706
Epoch 150 MSE:  0.303124338388443
Epoch 160 MSE:  0.2975121736526489
Epoch 170 MSE:  0.2903776466846466
Epoch 180 MSE:  0.28208521008491516
Epoch 190 MSE:  0.2750924527645111
Epoch 200 MSE:  0.2671164870262146
Epoch 210 MSE:  0.26062849164009094
Epoch 220 MSE:  0.25504839420318604
Epoch 230 MSE:  0.25122085213661194
Epoch 240 MSE:  0.2480061799287796
Epoch 250 MSE:  0.24589583277702332
Epoch 260 MSE:  0.2441454827785492
Epoch 270 MSE:  0.24296978116035461
Epoch 280 MSE:  0.24

In [11]:

with torch.no_grad():
    # make predictions
    model.eval()
    y_test_pred = model(x_test_tensor)


    y_train_pred_np = y_train_pred.detach().numpy().squeeze()
    y_test_pred_np = y_test_pred.detach().numpy().squeeze()

    print(y_train.shape)
    print(y_test.shape)
    print(y_train_pred_np.shape)
    print(y_test_pred_np.shape)

    # Calculate RMSE directly without inverse transformation
    trainScore = math.sqrt(mean_squared_error(y_train[timestep:], y_train_pred_np))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(y_test[timestep:], y_test_pred_np))
    print('Test Score: %.2f RMSE' % (testScore))

(12187,)
(5224,)
(12147,)
(5184,)
Train Score: 0.49 RMSE
Test Score: 0.50 RMSE


plot

In [12]:
signal = pd.Series(y_test[timestep:])
pred_sig = pd.Series(y_test_pred_np.flatten())
pred_sig.index = signal.index
combined_df = pd.concat([signal, pred_sig], axis=1)
fig = combined_df.vbt.plot()
fig.update_layout(yaxis_title='signal')
fig.show()

In [18]:
long_entries = pred_sig > 1.35
long_exits = pred_sig < 1
short_entries = pred_sig < 0.5
short_exits = pred_sig > 1.2
pf = vbt.Portfolio.from_signals(
    close=X_test.Close, 
    long_entries=long_entries, 
    long_exits=long_exits, 
    # short_entries=short_entries,
    # short_exits=short_exits,
    size=100,
    size_type='value',
    # accumulate=True,
    init_cash='auto'
)
pf.plot({"orders", "cum_returns"}, settings=dict(bm_returns=False)).show()


